# Sedaro Rendezous Tutorial Notebook 
Demonstrates using the Sedaro API  to calculate Hohmann Rendezous manevuers between two satelite using a Jupyter notebook.

Manuever 1: Start a Hohmann transfer to raise the orbit of the chaser satelite to the same altitude as the target satelite.
Manuever 2: Complete the Hohmann transfer to match the velocity of the target satelite.
Manuever 3: Engage the Richard WARAT (Wicked Awesome Rendezous Approach Thrust) algorithm 

In [1]:
import sedaro
import math
import yaml
import json
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current changes and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

## Sedaro python client setup
Note: More information about the sedaro-python client can be found here: [https://github.com/sedaro/sedaro-python]


In the next cell, adjust the following variables as needed
- *Sedaro_api_host*
- *Sedaro_api_token*

In [ ]:
Sedaro_api_host  = "http://localhost:80" # "api.sedaro.com"

# Set your API token value either directly or via loading a secrets file
# !! NOTE !!  Be careful not to check in your API Key into a source control repo  !! Note !!
secretPath = '/Users/sedaro/Documents/sedaro/sedaro-satellite/secrets.json'
with open(secretPath, 'r') as file:
    Sedaro_api_token = json.load(file)['API_KEY']
    
sedaroAPI = sedaro.SedaroApiClient(api_key=Sedaro_api_token, host=Sedaro_api_host)

Resources:
- Fundamentals of Astrodynamics 1971 (Bate, Mueller, White) pg 163, 362
- Fundamentals of Astrodynamics and Applications Fifth Edition 2022 (Vallado) pg 329, 367
 

In [39]:
# helper functions
def phase_angle_between_two_vectors(v1, v2):
    return 

def circuliar_orbit_radius_km(position):
    return 

def position_altitude(position):
    return 

Earth_radius_km = 6378.1363
earth_u_ER = 1
sec_per_TU = 806.811


In [113]:
# get the inputs we need to calculate the hohmann transfer
# 1. the current position of the chaser satellite
# 2. the current position of the target satellite


# workspace_id = "PLkM7LYDBLNQ2gpRvYvRTb"

# scenario_branch_id = "PLkNJHtS23GqvtxwkvwHHn"
# scenario_branch = sedaroAPI.scenario(scenario_branch_id)

# chaser_agent_branch_id = "PLkNJGQbsZSxf7l47SZHNH"
# chaser_agent_branch = sedaroAPI.agent_template(chaser_agent_branch_id)

# target_agent_branch_id = "PLkNJGQbsZSxf7l47SZHNH"
# target_agent_branch = sedaroAPI.agent_template(target_agent_branch_id)

# get the current position of the chaser satellite
#chaser_position = chaser_agent_branch.position
#chaser_radius_km = calc_circular_orbit_radius_km(chaser_position)
chaser_radius_km = 12756.274 # Vallado example pg 368

#target_position = target_agent_branch.position
#target_radius_km = calc_circular_orbit_radius_km(target_position)
target_radius_km = 42159.48 # Vallado example pg 368

# get the phase angle between the chaser and target
#phase_angle = phase_angle_between_two_vectors(chaser_position, target_position)
phase_angle_deg = -20.0 # Vallado example pg 368


In [137]:
# Circular Coplanar Phasing (different orbits)

def calc_radaii_canonical_units(radius_km):
    return radius_km / Earth_radius_km

def calc_circular_mean_motion(radius_ER):
    return math.sqrt(earth_u_ER / (radius_ER**3))


def calc_semi_major_axis_of_transfer_orbit(first_radius_ER, second_radius_ER):
    return (first_radius_ER + second_radius_ER) / 2

def calc_circular_orbit_velocity(radius_ER):
    return math.sqrt(earth_u_ER / radius_ER)

def calc_hohmann_transfer_dvs(chaser_radius_ER, target_radius_ER):
    # calculate the semi-major axis of the transfer orbit
    a_transfer = calc_semi_major_axis_of_transfer_orbit(chaser_radius_ER, target_radius_ER)

    # calculate the velocity of the chaser satellite in its current orbit
    v_chaser = calc_circular_orbit_velocity(chaser_radius_ER)

    # calculate the velocity of the target satellite in its current orbit
    v_target = calc_circular_orbit_velocity(target_radius_ER)

    # calculate the velocity of the chaser satellite at the start transfer orbit point
    v_start_transfer = (math.sqrt(earth_u_ER * ((2 / chaser_radius_ER) - (1 / a_transfer))))

    # calculate the velocity of the target satellite at the end _transfer orbit point
    v_end_transfer = (math.sqrt(earth_u_ER * ((2 / target_radius_ER) - (1 / a_transfer))))

    # calculate the velocity change required for the chaser satellite to enter the transfer orbit
    dv_to_enter_the_transfer_orbit = math.fabs(v_start_transfer - v_chaser)

    # calculate the velocity change required for the chaser satellite to exit the transfer orbit
    dv_to_exit_the_transfer_orbit = math.fabs(v_end_transfer - v_target ) 

    return dv_to_enter_the_transfer_orbit, dv_to_exit_the_transfer_orbit


def calc_hohmann_transfer_time(chaser_radius_ER, target_radius_ER):
    # calculate the semi-major axis of the transfer orbit
    a_transfer = calc_semi_major_axis_of_transfer_orbit(chaser_radius_ER, target_radius_ER)

    # calculate the period of the transfer orbit
    T_transfer =2 * math.pi * math.sqrt((a_transfer ** 3) / earth_u_ER)

    # calculate the time to transfer from the chaser orbit to the target orbit
    time_transfer = T_transfer / 2

    return time_transfer

def calc_lead_angle_for_target(chaser_radius_ER, target_radius_ER):
    target_mean_motion = calc_circular_mean_motion(target_radius_ER)
    return calc_hohmann_transfer_time(chaser_radius_ER, target_radius_ER) * target_mean_motion

def phasing_angle_for_target(chaser_radius_ER, target_radius_ER):
    return math.pi - calc_lead_angle_for_target(chaser_radius_ER, target_radius_ER)

def calc_hohmann_transfer_wait_time(chaser_radius_ER, target_radius_ER, phase_angle_deg, k):
    wait_time = (phasing_angle_for_target(chaser_radius_ER, target_radius_ER) - phase_angle_deg*math.pi/180 + 2*math.pi*k) / (calc_circular_mean_motion(chaser_radius_ER) - calc_circular_mean_motion(target_radius_ER))
    return wait_time


In [146]:
# valldo example pg 368
# convert the radii to canonical units
a_chaser_ER = calc_radaii_canonical_units(chaser_radius_km)
a_target_ER = calc_radaii_canonical_units(target_radius_km) 
assert a_target_ER, a_chaser_ER == (6.6099998521511685, 2.000000219499856)

In [145]:
wait_time_TU = calc_hohmann_transfer_wait_time(a_chaser_ER, a_target_ER, phase_angle_deg, k=1)
assert wait_time_TU == 27.561387230783627

In [144]:
dv_start, dv_end = calc_hohmann_transfer_dvs(a_chaser_ER, a_target_ER)
total_delta_v_ERperTU = dv_start + dv_end 
assert 0.29292887865895556 == total_delta_v_ERperTU

In [143]:
(dv_start + dv_end)*Earth_radius_km/sec_per_TU # km/s

2.3157100167114475